<a href="https://colab.research.google.com/github/scv718/test5/blob/master/3_%EB%82%A0%EC%94%A8_%EC%A0%95%EB%B3%B4_%EB%B0%8F_%EB%A7%9B%EC%A7%91_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##################################################
# 1. 날씨 정보 가져오기
##################################################

In [ ]:
# 1-1. 날씨 정보 가져오기
import requests
import datetime

vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"
service_key = "<YOUR SERVICE KEY>"

today = datetime.datetime.today()
base_date = today.strftime("%Y%m%d") # "20200214"
base_time = "0800"

# 첨부된 "기상청18_동네예보 조회서비스_오픈API활용가이드_격자_위경도.xlsx" 에서 참고한 위경도
nx = "60"
ny = "128"

payload = "serviceKey=" + service_key + "&" +\
    "dataType=json" + "&" +\
    "base_date=" + base_date + "&" +\
    "base_time=" + base_time + "&" +\
    "nx=" + nx + "&" +\
    "ny=" + ny

res = requests.get(vilage_weather_url + payload)

items = res.json()['response']['body']['items']['item']

In [ ]:
# 1-2. 날씨 정보 정제하기

# 날씨 정보를 담을 dictionary
data = dict()
data['date'] = base_date

weather_data = dict()
for item in items:
    # 기온
    if item['category'] == 'T3H':
        weather_data['tmp'] = item['fcstValue']
    
    # 기상상태
    if item['category'] == 'PTY':
        
        weather_code = item['fcstValue']
        
        if weather_code == '1':
            weather_state = '비'
        elif weather_code == '2':
            weather_state = '비/눈'
        elif weather_code == '3':
            weather_state = '눈'
        elif weather_code == '4':
            weather_state = '소나기'
        else:
            weather_state = '없음'
        
        weather_data['code'] = weather_code
        weather_data['state'] = weather_state

data['weather'] = weather_data
data['weather']

In [ ]:
##################################################
# 2. 미세먼지 정보 가져오기
##################################################

In [ ]:
# 2-1. 미세먼지 정보 가져오기

dust_url = "http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureLIst?"
#service_key는 "1-1. 날씨 정보 가져오기와 동일"
service_key = "<YOUR SERVICE KEY>"

item_code_pm10 = "PM10"
item_code_pm25 = "PM25"
data_gubun = "HOUR"
search_condition = "WEEK"

payload = "serviceKey=" + service_key + "&" +\
    "dataType=json" + "&" +\
    "dataGubun=" + data_gubun + "&" +\
    "searchCondition=" + search_condition  + "&" +\
    "itemCode="

# pm10 pm2.5 수치 가져오기
pm10_res = requests.get(dust_url + payload +item_code_pm10)
pm25_res = requests.get(dust_url + payload +item_code_pm25)

In [ ]:
# 2-2. 미세먼지 정보 정제하기

# xml 파싱하기
import xml.etree.ElementTree as elemTree

pm10_tree = elemTree.fromstring(pm10_res.text)
pm25_tree = elemTree.fromstring(pm25_res.text)

dust_data = dict()
for tree in [pm10_tree, pm25_tree]:
    item = tree.find("body").find("items").find("item")
    code = item.findtext("itemCode")
    value = int(item.findtext("seoul"))
    
    dust_data[code] = {'value' : value}

# 결과 값
dust_data

In [ ]:
# PM10 미세먼지 30 80 150
pm10_value = dust_data.get('PM10').get('value')
if pm10_value <= 30:
    pm10_state = "좋음"
elif pm10_value <= 80:
    pm10_state = "보통"
elif pm10_value <= 150:
    pm10_state = "나쁨"
else:
    pm10_state = "매우나쁨"

pm25_value = dust_data.get('PM2.5').get('value')
# PM2.5 초미세먼지 15 35 75
if pm25_value <= 15:
    pm25_state = "좋음"
elif pm25_value <= 35:
    pm25_state = "보통"
elif pm25_value <= 75:
    pm25_state = "나쁨"
else:
    pm25_state = "매우나쁨"

# 미세먼지가 나쁨 이상인지/아닌지
if pm10_value > 80 or  pm25_value > 75:
    dust_code = "1"
else:
    dust_code = "0"

dust_data.get('PM10')['state'] = pm10_state
dust_data.get('PM2.5')['state'] = pm25_state
dust_data['code'] = dust_code

data['dust'] = dust_data
data['dust']

NameError: ignored

In [ ]:
data = {'weather': {'code': '0', 'state': '없음', 'tmp': '9'},
 'date': '20200214',
 'dust': {'PM10': {'value': 94, 'state': '나쁨'},
  'PM2.5': {'value': 71, 'state': '나쁨'},
  'code': '1'}}

In [ ]:
##################################################
# 3. 날씨에 따른 음식 데이터 구하기
##################################################

In [ ]:
# 음식 정의
# 구글에서 검색하였습니다.
rain_foods = "부대찌개,아구찜,해물탕,칼국수,수제비,짬뽕,우동,치킨,국밥,김치부침개,두부김치,파전".split(',')
pmhigh_foods = "콩나물국밥,고등어,굴,쌀국수,마라탕".split(',')

In [ ]:
##################################################
# 4. 음식점 검색하기 - 네이버 검색 API
##################################################

In [ ]:
# 4-1. 네이버 인증
# https://developers.naver.com/apps
# 해당 사이트에서 로그인 후 "Cliend ID"와 "Client Secret"을 얻어오세요
ncreds = {
    "client_id": "<CLIENT_ID>",      
    "client_secret" : "<CLIENT_SECRET>"
}
nheaders = {
    "X-Naver-Client-Id" : ncreds.get('client_id'),
    "X-Naver-Client-Secret" : ncreds.get('client_secret')
}

In [ ]:
# 4-2. 날씨 조건 설정
# 경우 1 : 비/눈/소나기           => 비오는날 음식 3개 추천
# 경우 2 : 초/미세먼지 나쁨 이상  => 미세먼지에 좋은 음식 3개 추천
# 경우 3 : 정상                   => 블로그 리뷰 순 맛집 추천

# weather_state
if data.get('weather').get('code') != '0':
    weather_state = '1'
elif data.get('dust').get('code') == '1':
    weather_state = '2'
else:
    weather_state = '3'

In [ ]:
# 4-3. 조건에 해당하는 음식 리스트 뽑기
import random
# random.sample(x, k=len(x)) 무작위로 리스트 섞기

foods_list = None

# 경우 1, 2, 3
if weather_state == '1':
    foods_list = random.sample(rain_foods, k=len(rain_foods))
elif weather_state == '2':
    foods_list = random.sample(pmhigh_foods, k=len(pmhigh_foods))
else:
    foods_list = ['']

foods_list
# ['쌀국수', '굴', '콩나물국밥', '마라탕', '고등어']

In [ ]:
# 4-4. 네이버 맛집 검색하기
import urllib
# urllib.parse.quote(query) URL에서 검색어를 인코딩하기 위한 라이브러리

# 네이버 지역 검색 주소
naver_local_url = "https://openapi.naver.com/v1/search/local.json?"

# 검색에 사용될 파라미터
# 정렬 sort : 리뷰순(comment)
# 검색어 query : 인코딩된 문자열
params_format = "sort=comment&query="

# 위치는 사용자가 사용할 지역으로 변경가능
location = "국민대"

# 추천된 맛집을 담을 리스트
recommands = []
for food in foods_list:
    # 검색어 지정
    query = location + " " + food + " 맛집"
    enc_query = urllib.parse.quote(query)
    params = params_format + enc_query
    
    # 검색
    # headers : 네이버 인증 정보
    res = requests.get(naver_local_url + params, headers=nheaders)
    
    # 맛집 검색 결과
    result_list = res.json().get('items')

    # 경우 3 처리
    # 맛집 검색 결과에서 가장 상위 3개를 가져옴
    if weather_state == '3':
        recommands.append(result_list[:3])
        break
    
    # 경우 1,2 처리
    # 해당 음식 검색 결과에서 가장 상위를 가져옴
    if result_list:
        recommands.append(result_list[0])
        # 3개를 찾았다면 검색 중단
        if len(recommands) >= 3:
            break

In [ ]:
recommands

In [ ]:
##################################################
# 5. 추천 결과 카카오톡 보내기
##################################################

In [ ]:
# 5-1. 카카오톡 인증
# https://developers.kakao.com/docs/restapi/tool
# 해당 사이트에서 로그인 후 'Access token'을 얻어오세요
kcreds = {
    "access_token" : "<Acess token>"
}
kheaders = {
    "Authorization": "Bearer " + kcreds.get('access_token')
}

In [ ]:
# 5-2. 날씨 정보 카카오톡 보내기 - 텍스트 템플릿

# 카카오톡 URL 주소
kakaotalk_template_url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# 날씨 상세 정보 URL
weather_url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=%EB%82%A0%EC%94%A8"

# 날씨 정보 만들기
text = f"""\
#날씨 정보 ({data['date']})
기온 : {data['weather']['tmp']}
기우  : {data['weather']['state']}
미세먼지 : {data['dust']['PM10']['value']} {data['dust']['PM10']['state']}
초미세먼지 : {data['dust']['PM2.5']['value']} {data['dust']['PM2.5']['state']}
"""

# 텍스트 템플릿 형식 만들기
template = {
  "object_type": "text",
  "text": text,
  "link": {
    "web_url": weather_url,
    "mobile_web_url": weather_url
  },
  "button_title": "날씨 상세보기"
}

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.post(kakaotalk_template_url, data=payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))

In [ ]:
# 5-3. 음식 정보 보내기 - 리스트 템플릿

# 리스트 템플릿 형식 만들기
contents = []
template = {
    "object_type" : "list",
    "header_title" : "현재 날씨에 따른 음식 추천",
    "header_link" : {
        "web_url": weather_url,
        "mobile_web_url" : weather_url
    },
    "contents" : contents,
    "buttons" : [
        {
            "title" : "날씨 정보 상세보기",
            "link" : {
                "web_url": weather_url,
                "mobile_web_url" : weather_url
            }
        }
    ],
}

# contents 만들기
for place in recommands:
    title = place.get('title')  # 장소 이름
    # title : 태극쿵푸<b>마라탕</b>
    # html 태그 제거
    title = title.replace('<b>','').replace('</b>','')
    
    category = place.get('category')  # 장소 카테고리
    telephone = place.get('telephone')  # 장소 전화번호
    address = place.get('address')  # 장소 지번 주소

    # 각 장소를 클릭할 때 네이버 검색으로 연결해주기 위해 작성된 코드
    enc_address = urllib.parse.quote(address + ' ' + title)
    query = "query=" + enc_address

    # 장소 카테고리가 카페이면 카페 이미지
    # 이외에는 음식 이미지
    if '카페' in category:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    else:
        image_url = "https://freesvg.org/img/bentolunch.png?w=150&h=150&fit=fill"

    # 전화번호가 있다면 제목과 함께 넣어줍니다.
    if telephone:
        title = title + "\ntel) " + telephone

    # 카카오톡 리스트 템플릿 형식에 맞춰줍니다.
    content = {
        "title": "[" + category + "] " + title,
        "description": ' '.join(address.split()[1:]),
        "image_url": image_url,
        "image_width": 50, "image_height": 50,
        "link": {
            "web_url": "https://search.naver.com/search.naver?" + query,
            "mobile_web_url": "https://search.naver.com/search.naver?" + query
        }
    }
    
    contents.append(content)

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.post(kakaotalk_template_url, data=payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))

In [ ]:
data